In [ ]:
import pandas as pd
import plotly.express as px

# Leitura dos Dados

Origem: https://www.kaggle.com/datasets/sudalairajkumar/novel-corona-virus-2019-dataset

In [ ]:
dados_covid = pd.read_csv("covid_19_data.csv")

In [ ]:
dados_covid.head()

# Entendimento das Colunas

Da própria fonte de dados:

In [ ]:
dados_covid.info()

In [ ]:
dados_covid.describe()

# Tratamento dos Dados por Coluna

### ObservationDate

In [ ]:
dados_covid["ObservationDate"]

In [ ]:
# formato MM/DD/YYYY
dados_covid["ObservationDate"] = pd.to_datetime(dados_covid["ObservationDate"], format="%m/%d/%Y")

In [ ]:
dados_covid["ObservationDate"]

### Province/State

In [ ]:
# dados_covid["Province/State"].sort_values().unique()

**1 - Alguns possuem espaços antes ou depois**

In [ ]:
# STRIP: Remoção de Espaços do Início e Final de cada String
dados_covid["Province/State"] = dados_covid["Province/State"].str.strip()

**2 - Alguns possuem o estado junto - Verificar se existe duplicata**

In [ ]:
# dados_covid["Province/State"].sort_values().unique()

No segundo caso pode ser bom para identificar a origem específica. Vai depender do objetivo da análise. Se essa coluna realmente for só para province/state poderíamos criar uma nova que permitisse essas específicas e remover todos esses complementos da coluna atual.

In [ ]:
dados_covid["Province/State"] = dados_covid["Province/State"].replace({ 
    'Falkland Islands (Islas Malvinas)': 'Falkland Islands',
    'Falkland Islands (Malvinas)': 'Falkland Islands'
})

In [ ]:
dados_covid["Province/State"] = dados_covid["Province/State"].replace(
    to_replace=r'[ ]*\([^)]*\)', value='', regex=True
)

**3 - Dados nulos**

In [ ]:
len(dados_covid["Province/State"])

In [ ]:
dados_covid["Province/State"].isna().sum()

### Country/Region

**1 - Alguns possuem espaços antes ou depois**

In [ ]:
# sorted(dados_covid["Country/Region"].unique())

In [ ]:
# STRIP: Remoção de Espaços do Início e Final de cada String
dados_covid["Country/Region"] = dados_covid["Country/Region"].str.strip()

**2 - Replaces**

In [ ]:
# sorted(dados_covid["Country/Region"].unique())

In [ ]:
dados_covid["Country/Region"] = dados_covid["Country/Region"].replace({
    "('St. Martin',)": "St. Martin",
    "Bahamas, The": "Bahamas",
    'Gambia, The': 'Gambia'
})

**3 - Dados Nulos**

In [ ]:
dados_covid["Country/Region"].isna().sum()

### Last Update

In [ ]:
dados_covid[["Last Update", "ObservationDate"]].iloc[[0,500,1000,10000,13000]]

**1/22/2020 17:00**

In [ ]:
match_filter = dados_covid["Last Update"].str.match("[0-9]{1,2}\/[0-9]{1,2}\/[0-9]{4} [0-9]{1,2}:[0-9]{2}")

In [ ]:
dados_covid.loc[match_filter, "Last Update (ts)"] = pd.to_datetime(
    dados_covid["Last Update"].loc[match_filter], 
    format="%m/%d/%Y %H:%M"
)

**3/8/20 5:31**

In [ ]:
match_filter = dados_covid["Last Update"].str.match("[0-9]{1,2}\/[0-9]{1,2}\/[0-9]{2} [0-9]{1,2}:[0-9]{2}")

In [ ]:
dados_covid.loc[match_filter, "Last Update (ts)"] = pd.to_datetime(
    dados_covid["Last Update"].loc[match_filter], 
    format="%m/%d/%y %H:%M"
)

**2020-02-01T01:52:40**

In [ ]:
match_filter = dados_covid["Last Update"].str.match("[0-9]{4}-[0-9]{1,2}-[0-9]{1,2}T[0-9]{2}:[0-9]{2}:[0-9]{2}")

In [ ]:
dados_covid.loc[match_filter, "Last Update (ts)"] = pd.to_datetime(
    dados_covid["Last Update"].loc[match_filter], 
    format="%Y/%m/%dT%H:%M:%S"
)

**2020-02-01 01:52:40**

In [ ]:
match_filter = dados_covid["Last Update"].str.match("[0-9]{4}-[0-9]{1,2}-[0-9]{1,2} [0-9]{2}:[0-9]{2}:[0-9]{2}")

In [ ]:
dados_covid.loc[match_filter, "Last Update (ts)"] = pd.to_datetime(
    dados_covid["Last Update"].loc[match_filter], 
    format="%Y/%m/%d %H:%M:%S"
)

**Verificando se Atualizamos Todos**

In [ ]:
dados_covid["Last Update (ts)"].isna().sum()

**Atualizando a Coluna**

In [ ]:
dados_covid["Last Update"] = dados_covid["Last Update (ts)"]

In [ ]:
dados_covid.drop("Last Update (ts)", axis=1, inplace=True)

### Confirmed

In [ ]:
dados_covid["Confirmed"].dtype # ok

In [ ]:
dados_covid["Confirmed"].describe()

**Análise:** Estranho ter dados negativos.

In [ ]:
dados_covid["Confirmed"].plot.hist(bins=30)

In [ ]:
px.histogram(dados_covid["Confirmed"])

**Remoção de negativos**

In [ ]:
dados_covid[dados_covid["Confirmed"] < 0]

In [ ]:
indices_removidos = dados_covid[dados_covid["Confirmed"] < 0].index

In [ ]:
dados_covid.drop(index=indices_removidos, inplace=True)

### Deaths

In [ ]:
dados_covid["Deaths"].dtype # ok

In [ ]:
dados_covid["Deaths"].describe()

**Análise:** Estranho ter dados negativos.

In [ ]:
dados_covid["Deaths"].plot.hist(bins=30)

**Remoção de negativos**

In [ ]:
dados_covid[dados_covid["Deaths"] < 0]

In [ ]:
dados_covid.drop(index=dados_covid[dados_covid["Deaths"] < 0].index, inplace=True)

### Recovered

In [ ]:
dados_covid["Recovered"].dtype # ok

In [ ]:
dados_covid["Recovered"].describe()

**Análise:** Estranho ter dados negativos.

In [ ]:
dados_covid["Recovered"].plot.hist(bins=30)

**Remoção de negativos**

In [ ]:
dados_covid[dados_covid["Recovered"] < 0]

In [ ]:
dados_covid.drop(index=dados_covid[dados_covid["Recovered"] < 0].index, inplace=True)

### Dados da Colombia

Todos os dados estranhos são da colombia

In [ ]:
dados_colombia = dados_covid[dados_covid["Country/Region"] == "Colombia"]

In [ ]:
dados_colombia.pivot(index="ObservationDate", columns="Province/State", values="Confirmed")

In [ ]:
pivot_colombia = dados_colombia.pivot(index="ObservationDate", columns="Province/State", values="Confirmed")

In [ ]:
px.line(dados_colombia.pivot(index="ObservationDate", columns="Province/State", values="Confirmed"))

In [ ]:
px.line(dados_colombia.pivot(index="ObservationDate", columns="Province/State", values="Deaths"))

In [ ]:
px.line(dados_colombia.pivot(index="ObservationDate", columns="Province/State", values="Recovered"))

# Tratamento de Duplicatas

In [ ]:
dados_covid = dados_covid.drop_duplicates(subset=["ObservationDate", "Province/State", "Country/Region"], keep="last")

# Análise Brasil

In [ ]:
dados_brasil = dados_covid[dados_covid["Country/Region"] == "Brazil"]

In [ ]:
dados_brasil

In [ ]:
# Vamos apagar os dados nulos pois sao de quando nao dividia por estado
dados_brasil = dados_brasil.dropna()

In [ ]:
px.line(dados_brasil.pivot(index="ObservationDate", columns="Province/State", values="Deaths"))

## Interpolação de Séries

### Exemplo

In [ ]:
dados_rio = dados_brasil[dados_brasil["Province/State"] == "Rio de Janeiro"].set_index("ObservationDate").sort_index()
dados_rio

In [ ]:
sample_dados_rio = dados_rio.sample(int(len(dados_rio) * 0.1)).sort_index()

In [ ]:
sample_dados_rio["Confirmed"].plot(markeredgecolor="red", marker='x')

In [ ]:
sample_dados_rio["Confirmed"].asfreq('d').interpolate("linear").plot(markeredgecolor="red", marker='x')

# Correlação de dados

In [ ]:
pivot_covid = dados_covid.pivot_table(index="ObservationDate", columns="Country/Region", values="Confirmed", aggfunc="sum")

In [ ]:
paises_selecionados = ["Brazil", "Ecuador", "Chile", "Argentina", "US", "Canada", "UK", "Italy", "Mainland China", "Australia"]
pivot_selecionados = pivot_covid[paises_selecionados]

In [ ]:
pivot_selecionados

**Dados Nulos são preocupantes pois quebram a correlação**

In [ ]:
pivot_selecionados = pivot_selecionados.iloc[50:]

In [ ]:
corr_dados = pivot_selecionados.corr()

In [ ]:
corr_dados["Brazil"].sort_values().dropna()

In [ ]:
# Perceba que correlacao nao tem a ver com os valores brutos em si, mas com a relacao de crescimento

pivot_selecionados[["Brazil", "Ecuador"]].plot()

In [ ]:
# Correlacao ajustada

valores_finais = pivot_selecionados[["Brazil", "Ecuador"]].iloc[-1]

(pivot_selecionados[["Brazil", "Ecuador"]] / valores_finais).plot()

In [ ]:
# Correlacao ajustada

valores_finais = pivot_selecionados[["Brazil", "Australia"]].iloc[-1]

(pivot_selecionados[["Brazil", "Australia"]] / valores_finais).plot()

In [ ]:
# Perceba como a australia de fato tem uma curva diferente das demais

valores_iniciais = pivot_selecionados[paises_selecionados].iloc[0]
valores_finais = pivot_selecionados[paises_selecionados].iloc[-1]
( (pivot_selecionados[paises_selecionados] - valores_iniciais) / (valores_finais - valores_iniciais) ).plot()